In [59]:
import xarray as xr
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import cf_xarray.units  # noqa: F401
import pint_xarray  # noqa: F401
from datetime import datetime

pd.options.plotting.backend = "plotly"

---

## Load datasets


In [60]:
data_bats = xr.open_dataset(
    "../stations/bats/3_post_processed/bats_product.zarr", engine="zarr"
)
data_bats

<xarray.Dataset> Size: 16MB
Dimensions:                        (time: 4428, depth: 8, latitude: 1,
                                    longitude: 1, is_day: 2, sieve_size: 5)
Coordinates:
  * depth                          (depth) float64 64B 50.0 100.0 ... 400.0
  * is_day                         (is_day) bool 2B False True
  * latitude                       (latitude) float64 8B 31.6
  * longitude                      (longitude) float64 8B -64.2
  * sieve_size                     (sieve_size) float64 40B 0.2 0.5 1.0 2.0 5.0
  * time                           (time) datetime64[ns] 35kB 1988-10-21T04:5...
Data variables: (12/37)
    CO2                            (time, depth, latitude, longitude) float64 283kB ...
    CTD_salinity                   (time, depth, latitude, longitude) float64 283kB ...
    POC                            (time, depth, latitude, longitude) float64 283kB ...
    PON                            (time, depth, latitude, longitude) float64 283kB ...
    POP                            (time, depth, latitude, longitude) float64 283kB ...
    TN                             (time, depth, latitude, longitude) float64 283kB ...
    ...                             ...
    silicate                       (time, depth, latitude, longitude) float64 283kB ...
    synechococcus                  (time, depth, latitude, longitude) float64 283kB ...
    t0                             (time, depth, latitude, longitude) float64 283kB ...
    temp                           (time, depth, latitude, longitude) float64 283kB ...
    total_dissolved_phosphorus     (time, depth, latitude, longitude) float64 283kB ...
    wet_weight_vol_water_ratio     (time, is_day, depth, latitude, longitude, sieve_size) float64 3MB ...

In [61]:
data_hot = xr.open_dataset(
    "../stations/hot/3_post_processed/hot_product.zarr", engine="zarr"
)
data_hot

<xarray.Dataset> Size: 4MB
Dimensions:     (time: 687, is_day: 2, latitude: 1, longitude: 1, depth: 8,
                 sieve_size: 5)
Coordinates:
  * depth       (depth) int64 64B 50 100 150 200 250 300 350 400
  * is_day      (is_day) bool 2B False True
  * latitude    (latitude) float64 8B 22.75
  * longitude   (longitude) int64 8B -158
  * sieve_size  (sieve_size) float64 40B 0.2 0.5 1.0 2.0 5.0
  * time        (time) datetime64[ns] 5kB 1988-10-31 1988-12-02 ... 2022-09-02
Data variables: (12/25)
    abnd        (time, is_day, latitude, longitude, depth, sieve_size) float64 440kB ...
    bsal        (time, latitude, longitude, depth) float64 44kB ...
    carb        (time, is_day, latitude, longitude, depth, sieve_size) float64 440kB ...
    chl         (time, latitude, longitude, depth) float64 44kB ...
    d12         (time, latitude, longitude, depth) float64 44kB ...
    dwt         (time, is_day, latitude, longitude, depth, sieve_size) float64 440kB ...
    ...          ...
    svol        (time, is_day, latitude, longitude, depth, sieve_size) float64 440kB ...
    temp        (time, latitude, longitude, depth) float64 44kB ...
    theta       (time, latitude, longitude, depth) float64 44kB ...
    vol         (time, is_day, latitude, longitude, depth, sieve_size) float64 440kB ...
    wwt         (time, is_day, latitude, longitude, depth, sieve_size) float64 440kB ...
    xmiss       (time, latitude, longitude, depth) float64 44kB ...

In [62]:
data_papa = xr.open_dataset(
    "../stations/papa/3_post_processed/papa_product.zarr", engine="zarr"
)
data_papa.load()

<xarray.Dataset> Size: 36kB
Dimensions:           (time: 69, is_day: 2, latitude: 1, longitude: 1, depth: 4)
Coordinates:
  * depth             (depth) int64 32B 150 200 250 300
  * is_day            (is_day) bool 2B False True
  * latitude          (latitude) int64 8B 50
  * longitude         (longitude) int64 8B -150
  * time              (time) datetime64[ns] 552B 1997-02-21 ... 2020-08-20
Data variables:
    chaetognaths      (time, is_day, latitude, longitude, depth) float64 4kB ...
    crustaceans       (time, is_day, latitude, longitude, depth) float64 4kB ...
    foram             (time, is_day, latitude, longitude, depth) float64 4kB ...
    larvae            (time, is_day, latitude, longitude, depth) float64 4kB ...
    other             (time, is_day, latitude, longitude, depth) float64 4kB ...
    pteropods         (time, is_day, latitude, longitude, depth) float64 4kB ...
    small_gelatinous  (time, is_day, latitude, longitude, depth) float64 4kB ...
    thaliaceans       (time, is_day, latitude, longitude, depth) float64 4kB ...

---

## Vizualization

### BAT station


In [63]:
zoo_bats = (
    data_bats["dry_weight_vol_water_ratio"]
    .dropna("time", how="all")
    .mean("depth")
    .isel(latitude=0, longitude=0)
    .sum("sieve_size")
)
zoo_bats_d_n = zoo_bats.to_dataframe().pivot_table(
    index=["time"], columns=["is_day"], values=["dry_weight_vol_water_ratio"]
)
zoo_bats_d_n.columns = ["night", "day"]
# transform 0 to nan
zoo_bats_d_n = zoo_bats_d_n.map(lambda x: None if x == 0 else x)
zoo_bats_d_n

,night,day
time,,
1995-05-10,2.770,NaN
1995-05-11,NaN,2.410
1995-06-13,2.210,1.780
1995-06-14,3.370,NaN
1995-07-11,3.145,2.605
...,...,...
2022-09-17,1.780,NaN
2022-09-18,1.875,NaN
2022-11-23,1.465,NaN


In [64]:
day = zoo_bats_d_n["day"].dropna()
night = zoo_bats_d_n["night"].dropna()
go.Figure(
    data=[
        go.Scatter(x=day.index, y=day, mode="lines", name="Day"),
        go.Scatter(x=night.index, y=night, mode="lines", name="Night"),
    ]
).update_layout(
    title="BATS : zooplankton dry weight by cube meter of sea water",
    xaxis_title="Time",
    yaxis_title="Dry Weight (mg/m3)",
    legend_title="Time of Day",
).show()

### HOT station


In [65]:
zoo_hot = (
    data_hot["dwt"]
    .dropna("time", how="all")
    .mean("depth")
    .isel(latitude=0, longitude=0)
    .sum("sieve_size")
)
zoo_hot = zoo_hot.pint.quantify("g/m3").pint.to("mg/m^3").pint.dequantify()
zoo_hot_d_n = zoo_hot.to_dataframe().pivot_table(
    index=["time"], columns=["is_day"], values=["dwt"]
)
zoo_hot_d_n.columns = ["night", "day"]
# # transform 0 to nan
zoo_hot_d_n = zoo_hot_d_n.map(lambda x: None if x == 0 else x)
zoo_hot_d_n

,night,day
time,,
1994-02-17,3.699338,1.051934
1994-02-18,3.925000,0.604762
1994-02-19,3.259854,3.321583
1994-03-09,2.529457,2.003175
1994-03-10,3.760984,1.545122
...,...,...
2022-07-10,10.020219,7.180867
2022-07-30,13.231735,NaN
2022-07-31,9.892350,5.472701


In [66]:
day = zoo_hot_d_n["day"].dropna()
night = zoo_hot_d_n["night"].dropna()
go.Figure(
    data=[
        go.Scatter(x=day.index, y=day, mode="lines", name="Day"),
        go.Scatter(x=night.index, y=night, mode="lines", name="Night"),
    ]
).update_layout(
    title="HOT : zooplankton dry weight by cube meter of sea water",
    xaxis_title="Time",
    yaxis_title="Dry Weight (g/m3)",
    legend_title="Time of Day",
).show()

### PAPA station


In [67]:
# data = data_papa.to_dataframe().droplevel([2, 3], axis=0)
zoo_papa_d_n = (
    data_papa.mean("depth")
    .isel(latitude=0, longitude=0)
    .to_dataframe()
    .replace(0, None)
)
# enlève les colonnes "latitude" et "longitude"
zoo_papa_d_n = zoo_papa_d_n.drop(["latitude", "longitude"], axis=1)
zoo_papa_d_n = (
    zoo_papa_d_n.stack()
    .rename("zoo dry weight")
    .to_frame()
    .rename_axis(["time", "is_day", "species"])
)
zoo_papa_d_n = zoo_papa_d_n.reset_index()
zoo_papa_d_n

,time,is_day,species,zoo dry weight
0,1997-02-21,False,chaetognaths,0.30098
1,1997-02-21,False,crustaceans,5.353207
2,1997-02-21,False,foram,0.005687
3,1997-02-21,False,larvae,0.22576
4,1997-02-21,False,other,0.22645
...,...,...,...,...
522,2020-08-20,False,larvae,0.15629
523,2020-08-20,False,other,0.53678
524,2020-08-20,False,pteropods,0.45086
525,2020-08-20,False,small_gelatinous,3.90876


In [68]:
fig = px.area(
    zoo_papa_d_n,
    x="time",
    y="zoo dry weight",
    line_group="species",
    color="species",
    facet_row="is_day",
)
fig.update_layout(
    title="PAPA : zooplankton dry weight by cube meter of sea water",
    xaxis_title="Time",
    yaxis_title="Dry Weight (mg/m3)",
    legend_title="Species",
)

In [69]:
# fig.write_html("zoo_dry_weight.html")

We remove the chaetognaths from the dataset, as they are not feeding on phytoplankton.


In [70]:
zoo_papa_d_n = (
    zoo_papa_d_n[zoo_papa_d_n["species"] != "chaetognaths"]
    .groupby(["time", "is_day"])["zoo dry weight"]
    .sum()
    .to_frame()
)
zoo_papa_d_n = zoo_papa_d_n.reset_index().pivot(
    index="time", columns="is_day", values="zoo dry weight"
)
zoo_papa_d_n.columns = ["night", "day"]
zoo_papa_d_n = zoo_papa_d_n.reset_index()
zoo_papa_d_n

,time,night,day
0,1997-02-21,6.81742,NaN
1,1997-06-15,10.81457,NaN
2,1997-06-16,NaN,20.694777
3,1997-09-04,NaN,10.367363
4,1997-09-05,NaN,6.42334
...,...,...,...
64,2019-02-12,4.86593,NaN
65,2019-06-11,NaN,10.70547
66,2019-08-23,NaN,3.47155
67,2020-02-16,NaN,5.451


In [71]:
day = zoo_papa_d_n["day"].dropna()
night = zoo_papa_d_n["night"].dropna()
go.Figure(
    data=[
        go.Scatter(x=day.index, y=day, mode="lines", name="Day"),
        go.Scatter(x=night.index, y=night, mode="lines", name="Night"),
    ]
).update_layout(
    title="HOT : zooplankton dry weight by cube meter of sea water",
    xaxis_title="Time",
    yaxis_title="Dry Weight (g/m3)",
    legend_title="Time of Day",
).show()

---

## Export the products in netCDF format


In [87]:
for df_zoo, name_zoo in zip(
    [zoo_hot_d_n, zoo_bats_d_n, zoo_papa_d_n], ["hot", "bats", "papa"]
):
    as_xarray = xr.Dataset.from_dataframe(df_zoo)
    as_xarray["time"].attrs = {
        "axis": "T",
        "standard_name": "time",
        "long_name": "time",
    }
    as_xarray["night"].attrs = {
        "units": "mg/m3",
        "description": "Zooplankton dry weight by cube meter of sea water at night",
    }
    as_xarray["day"].attrs = {
        "units": "mg/m3",
        "description": "Zooplankton dry weight by cube meter of sea water during the day",
    }
    as_xarray.attrs = {"station": name_zoo, "creation date": datetime.now().isoformat()}
    as_xarray.to_netcdf(f"{name_zoo}_zooplankton.nc")